<a href="https://colab.research.google.com/github/xzdil/googlecollabprojects/blob/main/SaigaMistral_llama_index(V%3F).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget https://huggingface.co/TheBloke/saiga_mistral_7b-GGUF/resolve/main/saiga_mistral_7b.Q4_K_M.gguf
!wget https://huggingface.co/TheBloke/saiga_mistral_7b-GGUF/resolve/main/saiga_mistral_7b.Q5_K_S.gguf

%pip install llama-index-embeddings-huggingface
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-index-llms-llama-cpp
!pip install llama-index gradio
%pip install --upgrade --quiet  docx2txt

In [3]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt
)

In [ ]:
!nvidia-smi

Fri Mar 29 12:34:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
model_url = 'https://huggingface.co/TheBloke/saiga_mistral_7b-GGUF/resolve/main/saiga_mistral_7b.Q4_K_M.gguf'

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='/content/saiga_mistral_7b.Q5_K_S.gguf',
    max_new_tokens=4000,
    context_window=20000,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1,
                  "cache":True,
                  "use_mmap":True},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

def gen_stream(text):
  response_iter = llm.stream_complete(text)
  for response in response_iter:
      print(response, end="", flush=True)

#gen_stream("Как сварить яйца")

In [ ]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
import requests

url = "https://www.dropbox.com/scl/fi/vnbhfi2eu1ciuwjwzsnnn/QA.docx?rlkey=3dpojwnrpt4f34q9gx461yode&dl=1"
file_path = "docs/QA.docx"

response = requests.get(url)
response.raise_for_status()

!mkdir docs
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"Файл успешно скачан и сохранен как {file_path}")

Файл успешно скачан и сохранен как docs/QA.docx


In [9]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('/content/docs').load_data()

In [10]:
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [17]:
query_engine = index.as_query_engine(llm=llm,streaming=True)

In [ ]:
# response = query_engine.query("Как мне себя обезопасить при покупке строящегося жилья")
# for text in response.response_gen:
#     print(text, end='',flush=True)

In [33]:
MODEL_NAME = "Saiga Mistal 7B GPTQ"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Ассистент по договору о долевом участии, отвечай строго по инструкции, если вопрос задается не о договоре о долевом участии, проси задавать вопросы только по теме."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [34]:
def chat():
  while True:
    inp = input()
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt()

    for token in query_engine.query(prompt).response_gen:
      print(token, end="", flush=True)
    print()
    print("="*150)
    print()

In [35]:
chat()

Почему трава зеленая


Llama.generate: prefix-match hit


 Зеленый цвет травы обуславливается наличием в ней хлорофилла, который является пигментом, отвечающим за фотосинтез. Фотосинтез - это процесс, благодаря которому растения получают энергию из света и углекислого газа, а также преобразуют их в питательные вещества для своего роста и развития. Хлорофилл поглощает световые волны длиной 430-450 нм (синий цвет) и 670-680 нм (красный цвет), что обуславливает зеленый цвет травы.


llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =     105.46 ms /   186 runs   (    0.57 ms per token,  1763.72 tokens per second)
llama_print_timings: prompt eval time =     651.67 ms /   503 tokens (    1.30 ms per token,   771.86 tokens per second)
llama_print_timings:        eval time =    6022.40 ms /   185 runs   (   32.55 ms per token,    30.72 tokens per second)
llama_print_timings:       total time =    7590.56 ms /   688 tokens


KeyboardInterrupt: Interrupted by user

In [38]:
def gen(text, history):
  print(history)
  conversation = Conversation()
  conversation.add_user_message(text)
  prompt = conversation.get_prompt()
  chat = ''
  for token in query_engine.query(prompt).response_gen:
      chat += token
      yield chat


In [ ]:
import gradio as gr
demo = gr.ChatInterface(fn=gen, examples=["Что такое договор о долевом участии?",
                                          "Чем отличается уполномоченная компания от застройщика?",
                                          "Как мне себя обезопасить при покупке строящегося жилья?",
                                          "Где я могу найти утвержденную форму договора долевого участия?",
                                          "Какие действия необходимо совершить дольщику для приобретения жилья в строящемся доме?",
                                          "Кто гарантирует завершение строительства жилого объекта?",
                                          "Какие меры предпринимаются для предотвращения риска двойных продаж недвижимости?",
                                          "Какие договора не безопасны?"],
                        title="ДелУчас-бот",
                        description="Это чат-бот, у которого вы можете спросить информацию о делевом участии в жилищном строительстве в РК")
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://34cfecd9ce55f59f40.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]


Llama.generate: prefix-match hit

llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =      22.10 ms /    40 runs   (    0.55 ms per token,  1809.95 tokens per second)
llama_print_timings: prompt eval time =     327.79 ms /    14 tokens (   23.41 ms per token,    42.71 tokens per second)
llama_print_timings:        eval time =    1148.81 ms /    39 runs   (   29.46 ms per token,    33.95 tokens per second)
llama_print_timings:       total time =    1647.11 ms /    53 tokens


[['Привет', ' Привет! Я могу помочь вам с любыми вопросами по договору о долевом участии в жилищном строительстве.']]


Llama.generate: prefix-match hit

llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =      43.14 ms /    77 runs   (    0.56 ms per token,  1785.05 tokens per second)
llama_print_timings: prompt eval time =     624.64 ms /   512 tokens (    1.22 ms per token,   819.67 tokens per second)
llama_print_timings:        eval time =    2190.45 ms /    76 runs   (   28.82 ms per token,    34.70 tokens per second)
llama_print_timings:       total time =    3152.18 ms /   588 tokens


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =      23.04 ms /    40 runs   (    0.58 ms per token,  1736.11 tokens per second)
llama_print_timings: prompt eval time =    1174.45 ms /   978 tokens (    1.20 ms per token,   832.73 tokens per second)
llama_print_timings:        eval time =    1159.72 ms /    39 runs   (   29.74 ms per token,    33.63 tokens per second)
llama_print_timings:       total time =    2511.23 ms /  1017 tokens


[['Привет', ' Привет! Я могу помочь вам с любыми вопросами по договору о долевом участии в жилищном строительстве.']]


Llama.generate: prefix-match hit

llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =      27.84 ms /    48 runs   (    0.58 ms per token,  1724.45 tokens per second)
llama_print_timings: prompt eval time =     622.16 ms /   510 tokens (    1.22 ms per token,   819.72 tokens per second)
llama_print_timings:        eval time =    1357.31 ms /    47 runs   (   28.88 ms per token,    34.63 tokens per second)
llama_print_timings:       total time =    2203.89 ms /   557 tokens


[['Привет', ' Привет! Я могу помочь вам с любыми вопросами по договору о долевом участии в жилищном строительстве.'], ['что я сказал пару сообщений назад?', ' \nВы сказали: "Какие преимущества обладает Единый оператор в лице АО «Казахстанской Жилищной Компании»?"']]


Llama.generate: prefix-match hit

llama_print_timings:        load time =     672.36 ms
llama_print_timings:      sample time =      32.91 ms /    57 runs   (    0.58 ms per token,  1732.26 tokens per second)
llama_print_timings: prompt eval time =     365.65 ms /    24 tokens (   15.24 ms per token,    65.64 tokens per second)
llama_print_timings:        eval time =    1619.39 ms /    56 runs   (   28.92 ms per token,    34.58 tokens per second)
llama_print_timings:       total time =    2250.06 ms /    80 tokens


In [ ]:
from llama_index.core import PromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.chat_engine import CondenseQuestionChatEngine

custom_prompt = PromptTemplate(
    """\
Учитывая разговор (между Человеком и Помощником) и последующее сообщение от Человека, \
перепишите сообщение, чтобы оно стало отдельным вопросом, охватывающим весь соответствующий контекст \
из разговора.

<История чата>
{chat_history}

<Последующее сообщение>
{question}

<Отдельный вопрос>
"""
)

custom_chat_history = []
chat_engine = CondenseQuestionChatEngine.from_defaults(
      llm=llm,
      query_engine=query_engine,
      condense_question_prompt=custom_prompt,
      chat_history=custom_chat_history,
      verbose=True,
      streaming=True
  )

In [ ]:
def chat(message, history):
  print(history)
  streaming_response = chat_engine.stream_chat(message)
  response = ''
  for token in streaming_response.response_gen:
      response += token
      yield response
  custom_chat_history.append(ChatMessage(role=MessageRole.USER, content = message))
  custom_chat_history.append(ChatMessage(role=MessageRole.ASSISTANT, content = response))

In [ ]:
# def custom_clear_fn(history):
#   history = []
#   return history

In [ ]:
import gradio as gr
custom_chat_history.clear()
demo = gr.ChatInterface(fn=chat, examples=["Что такое договор о долевом участии?",
                                          "Чем отличается уполномоченная компания от застройщика?",
                                          "Как мне себя обезопасить при покупке строящегося жилья?",
                                          "Какие способы покупки строящего жилья допустимы?",
                                          "Какие договора не безопасны?"],
                        title="ДелУчас-бот",
                        description="Это чат-бот, у которого вы можете спросить информацию о делевом участии в жилищном строительстве в РК")

# custom_button = gr.Button("Custom Button")
# custom_button.observe(custom_clear_fn, custom_chat_history)
# demo.chat_interface.add_element(custom_button)
# demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2963ff4b8fe1cd8641.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]



llama_print_timings:        load time =     347.72 ms
llama_print_timings:      sample time =      18.92 ms /    28 runs   (    0.68 ms per token,  1479.68 tokens per second)
llama_print_timings: prompt eval time =    2668.46 ms /   200 tokens (   13.34 ms per token,    74.95 tokens per second)
llama_print_timings:        eval time =     687.42 ms /    28 runs   (   24.55 ms per token,    40.73 tokens per second)
llama_print_timings:       total time =    3544.62 ms /   228 tokens
Llama.generate: prefix-match hit


Querying with: 
Как мне себя обезопасить при покупке строящегося жилья?



llama_print_timings:        load time =     347.72 ms
llama_print_timings:      sample time =    1672.19 ms /  1538 runs   (    1.09 ms per token,   919.75 tokens per second)
llama_print_timings: prompt eval time =   15454.49 ms /  1200 tokens (   12.88 ms per token,    77.65 tokens per second)
llama_print_timings:        eval time =   45426.92 ms /  1537 runs   (   29.56 ms per token,    33.83 tokens per second)
llama_print_timings:       total time =   85634.24 ms /  2737 tokens


...

Querying with: 
Какие договора не безопасны при покупке строящегося жилья?



llama_print_timings:        load time =     347.72 ms
llama_print_timings:      sample time =    1482.13 ms /  1129 runs   (    1.31 ms per token,   761.74 tokens per second)
llama_print_timings: prompt eval time =   17401.27 ms /  1200 tokens (   14.50 ms per token,    68.96 tokens per second)
llama_print_timings:        eval time =   35703.19 ms /  1128 runs   (   31.65 ms per token,    31.59 tokens per second)
llama_print_timings:       total time =   73016.16 ms /  2328 tokens


[['Как мне себя обезопасить при покупке строящегося жилья?', '\n\nВы можете защитить себя при покупке строящегося жилья следующим образом:\n\n1. Проверьте документы и лицензии строительной компании. Убедитесь, что компания имеет все необходимые лицензии и разрешения на строительство жилья.\n2. Проверьте проектное решение и техническое задание. Убедитесь, что проектное решение и техническое задание соответствуют требованиям и стандартам.\n3. Проверьте финансовые документы и планы строительства. Убедитесь, что строительство будет осуществляться в рамках бюджета и сроков.\n4. Проверьте контракт на долю в жилищном строительстве. Убедитесь, что все условия контракта соответствуют вашим ожиданиям и интересам.\n5. Проверьте гарантийные условия и страховые полисы. Убедитесь, что гарантийные условия и страховые полисы обеспечивают защиту ваших интересов в случае возникновения проблем с жильем.\n6. Проверьте условия и условия обслуживания жилья. Убедитесь, что условия обслуживания жилья соответс

Llama.generate: prefix-match hit

llama_print_timings:        load time =     347.72 ms
llama_print_timings:      sample time =       0.66 ms /     1 runs   (    0.66 ms per token,  1526.72 tokens per second)
llama_print_timings: prompt eval time =   43488.01 ms /  2853 tokens (   15.24 ms per token,    65.60 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   44403.15 ms /  2854 tokens
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 501, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 258, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1684, in process_api
    result = await self.call_function(
  File "/usr/lo

Querying with: 
